In [1]:
from external.coingecko.read_info_api import get_prices
from external.symbiotic.read_info_api import get_operators,get_networks,get_vaults
from external.symbiotic.read_info_cli import get_staking_data
from external.coingecko.read_info_api import get_prices
import polars as pl
from itertools import batched
import polars as pl
pl.Config.set_tbl_cols(50)
pl.Config.set_tbl_rows(20)  # Change 50 to any number

polars.config.Config

In [3]:
### Get data from Symbiotic API
df_operators = await get_operators()
df_networks = await get_networks()
#df_vaults = await get_vaults()

#df_staking_data = get_staking_data("python symb.py nets")

# Get unique addresses
unique_price_list = df_staking_data["collateral_asset_address"].unique().to_list()

# Store results in a list and convert to DataFrame at the end
price_results = []

for batch in batched(unique_price_list, 5):  # Batch size of 5
    contract_addresses_string = ",".join(batch)    
    price_results.append(await get_prices(contract_addresses_string))

# Convert the list of results into a Polars DataFrame
#prices = pl.concat(price_results)

#print(prices)

df_staking_data = df_staking_data.with_columns(
    pl.col("collateral_asset_address").str.to_lowercase().alias("collateral_asset_address")
)

df_staking_data = df_staking_data.join(
                        df_operators,
                        on=['node_operator_address'],
                        how='left'
                    ).join(
                        df_networks,
                        on=['network_address'],
                        how='left'
                        ).join(
                        prices,
                        on=['collateral_asset_address'],
                        how='left'
                    )

df_staking_data = df_staking_data.with_columns(
    pl.col("amount_staked") * pl.col("collateral_asset_price").alias("amount_staked_usd")
)
print(df_staking_data)
print(df_staking_data.null_count())

Getting operators from Symbiotic API
✅ Operators fetched successfully
Getting networks from Symbiotic API
✅ Networks fetched successfully
shape: (445, 9)
┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ network_ ┆ node_ope ┆ collater ┆ collater ┆ amount_s ┆ timestam ┆ node_ope ┆ network_ ┆ collater │
│ address  ┆ rator_ad ┆ al_asset ┆ al_asset ┆ taked    ┆ p        ┆ rator_na ┆ name     ┆ al_asset │
│ ---      ┆ dress    ┆ _address ┆ _symbol  ┆ ---      ┆ ---      ┆ me       ┆ ---      ┆ _price   │
│ str      ┆ ---      ┆ ---      ┆ ---      ┆ f64      ┆ datetime ┆ ---      ┆ str      ┆ ---      │
│          ┆ str      ┆ str      ┆ str      ┆          ┆ [μs]     ┆ str      ┆          ┆ f64      │
╞══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 0x9101ed ┆ 0xac128a ┆ 0xf951e3 ┆ sweth    ┆ 8262.6   ┆ 2025-03- ┆ null     ┆ Mev-comm ┆ 2065.65  │
│ a106a443 ┆ a884c64c ┆ 35afb289 ┆    

In [ ]:
df_staking_data = df_staking_data.drop(["collateral_asset_price", "node_operator_name","network_name"])

In [4]:
df_staking_data

network_address,node_operator_address,collateral_asset_address,collateral_asset_symbol,amount_staked,timestamp,node_operator_name,network_name,collateral_asset_price
str,str,str,str,f64,datetime[μs],str,str,f64
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xf951E335afb289353dc249e82926…","""sweth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xBe9895146f7AF43049ca1c1AE358…","""cbeth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xa2E3356610840701BDf5611a5397…","""wbeth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xae78736Cd615f374D3085123A210…","""reth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0x7f39C581F595B53c5cb19bD0b3f8…","""wsteth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0x9D39A5DE30e57443BfF2A8307A42…","""susde""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0x6f64decffde23aa598724325202f…","""0x7f39C581F595B53c5cb19bD0b3f8…","""wsteth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0x9321d38c355d1d8cb9dbfc05a5c0…","""0x8c1BEd5b9a0928467c9B1341Da1D…","""lseth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null
"""0x9101eda106a443a0fa82375936d0…","""0xd741f826c60dacebb8835278e0f1…","""0xf951E335afb289353dc249e82926…","""sweth""",null,2025-03-16 17:00:00,null,"""Mev-commit""",null


In [6]:
df_staking_data = df_staking_data.with_columns(
    (pl.col("amount_staked") * pl.col("collateral_asset_price")).alias("amount_staked_usd")
)


In [19]:
asset_decimals = 18
max_amount_staked = 1_000_000_000

df_staking_data = df_staking_data.with_columns(
        pl.when(pl.col("amount_staked") > max_amount_staked)
        .then(pl.col("amount_staked") / 10 ** asset_decimals)
        .otherwise(pl.col("amount_staked"))
        .alias("amount_staked")
    )

In [25]:
df_staking_data.filter(pl.col("node_operator_name") == "Other")

network_address,node_operator_address,collateral_asset_address,collateral_asset_symbol,amount_staked,timestamp,node_operator_name,network_name,collateral_asset_price,amount_staked_usd,operator_name
str,str,str,str,f64,datetime[μs],str,str,f64,f64,str
"""0x3a7b173124dcfecff1847ff7f8f5…","""0xcb87997ffc85d66df1e919d37c93…","""0x57b946008913b82e4df85f501cba…","""pond""",8777.509015,2025-03-16 18:00:00,"""Other""","""Kalypso""",0.011003,96.580161,null
"""0xfca0128a19a5c06b0148c27ee762…","""0xae41878469d385fada7c07f74c48…","""0xf951e335afb289353dc249e82926…","""sweth""",1.8238e-9,2025-03-16 18:00:00,"""Other""","""Radius""",2065.65,3.7674e12,null
"""0xfca0128a19a5c06b0148c27ee762…","""0xae41878469d385fada7c07f74c48…","""0x7f39c581f595b53c5cb19bd0b3f8…","""wsteth""",2.3538e-9,2025-03-16 18:00:00,"""Other""","""Radius""",2269.24,5.3413e12,null
"""0xfca0128a19a5c06b0148c27ee762…","""0xae41878469d385fada7c07f74c48…","""0xbe9895146f7af43049ca1c1ae358…","""cbeth""",3.8886e8,2025-03-16 18:00:00,"""Other""","""Radius""",2078.61,8.0828e11,null
"""0xfca0128a19a5c06b0148c27ee762…","""0x00e769e7067143fbdb962258e78f…","""0x2260fac5e5542a773aa44fbcfedf…","""wbtc""",2.7632e-8,2025-03-16 18:00:00,"""Other""","""Radius""",83772.0,2.3148e15,null


In [28]:
temp = df_staking_data.filter(pl.col("amount_staked_usd").is_not_null())
temp


network_address,node_operator_address,collateral_asset_address,collateral_asset_symbol,amount_staked,timestamp,node_operator_name,network_name,collateral_asset_price,amount_staked_usd,operator_name
str,str,str,str,f64,datetime[μs],str,str,f64,f64,str
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xf951e335afb289353dc249e82926…","""sweth""",1.7068e7,2025-03-16 18:00:00,"""Luganodes""","""Mev-commit""",2065.65,3.5256e10,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xbe9895146f7af43049ca1c1ae358…","""cbeth""",1.7282e7,2025-03-16 18:00:00,"""Luganodes""","""Mev-commit""",2078.61,3.5924e10,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xa2e3356610840701bdf5611a5397…","""wbeth""",1.6481e7,2025-03-16 18:00:00,"""Luganodes""","""Mev-commit""",2029.85,3.3454e10,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xae78736cd615f374d3085123a210…","""reth""",1.8308e7,2025-03-16 18:00:00,"""Luganodes""","""Mev-commit""",2139.38,3.9167e10,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0x7f39c581f595b53c5cb19bd0b3f8…","""wsteth""",4.7882e-8,2025-03-16 18:00:00,"""Luganodes""","""Mev-commit""",2269.24,1.0865e14,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0x9d39a5de30e57443bff2a8307a42…","""susde""",18838.4,2025-03-16 18:00:00,"""Luganodes""","""Mev-commit""",1.16,21852.544,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0x6f64decffde23aa598724325202f…","""0x7f39c581f595b53c5cb19bd0b3f8…","""wsteth""",4.6211e7,2025-03-16 18:00:00,"""Everstake""","""Mev-commit""",2269.24,1.0486e11,"""Everstake"""
"""0x9101eda106a443a0fa82375936d0…","""0x9321d38c355d1d8cb9dbfc05a5c0…","""0x8c1bed5b9a0928467c9b1341da1d…","""lseth""",3.7443e-9,2025-03-16 18:00:00,"""NodeInfra""","""Mev-commit""",2039.19,7.6353e12,"""NodeInfra"""
"""0x9101eda106a443a0fa82375936d0…","""0xd741f826c60dacebb8835278e0f1…","""0xf951e335afb289353dc249e82926…","""sweth""",1.7068e7,2025-03-16 18:00:00,"""Chorus One""","""Mev-commit""",2065.65,3.5256e10,"""Chorus One"""


In [34]:
df_staking_data.filter(pl.col("collateral_asset_price").is_null())

network_address,node_operator_address,collateral_asset_address,collateral_asset_symbol,amount_staked,timestamp,node_operator_name,network_name,collateral_asset_price,amount_staked_usd,operator_name
str,str,str,str,f64,datetime[μs],str,str,f64,f64,str


In [23]:
df_staking_data = df_staking_data.with_columns(
        pl.when(pl.col("node_operator_name").is_null())
        .then(pl.lit("Other"))
        .otherwise(pl.col("node_operator_name"))
        .alias("node_operator_name")
    )

In [14]:
df_staking_data

network_address,node_operator_address,collateral_asset_address,collateral_asset_symbol,amount_staked,timestamp,node_operator_name,network_name,collateral_asset_price,amount_staked_usd,operator_name
str,str,str,str,f64,datetime[μs],str,str,f64,f64,str
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xf951e335afb289353dc249e82926…","""sweth""",1.7068e7,2025-03-16 18:00:00,null,"""Mev-commit""",2065.65,3.5256e10,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xbe9895146f7af43049ca1c1ae358…","""cbeth""",1.7282e7,2025-03-16 18:00:00,null,"""Mev-commit""",2078.61,3.5924e10,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xa2e3356610840701bdf5611a5397…","""wbeth""",1.6481e7,2025-03-16 18:00:00,null,"""Mev-commit""",2029.85,3.3454e10,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0xae78736cd615f374d3085123a210…","""reth""",1.8308e7,2025-03-16 18:00:00,null,"""Mev-commit""",2139.38,3.9167e10,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0x7f39c581f595b53c5cb19bd0b3f8…","""wsteth""",4.7882e10,2025-03-16 18:00:00,null,"""Mev-commit""",2269.24,1.0865e14,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0xac128aa884c64cbe6afecf5c006d…","""0x9d39a5de30e57443bff2a8307a42…","""susde""",18838.4,2025-03-16 18:00:00,null,"""Mev-commit""",1.16,21852.544,"""Luganodes"""
"""0x9101eda106a443a0fa82375936d0…","""0x6f64decffde23aa598724325202f…","""0x7f39c581f595b53c5cb19bd0b3f8…","""wsteth""",4.6211e7,2025-03-16 18:00:00,null,"""Mev-commit""",2269.24,1.0486e11,"""Everstake"""
"""0x9101eda106a443a0fa82375936d0…","""0x9321d38c355d1d8cb9dbfc05a5c0…","""0x8c1bed5b9a0928467c9b1341da1d…","""lseth""",3.7443e9,2025-03-16 18:00:00,null,"""Mev-commit""",2039.19,7.6353e12,"""NodeInfra"""
"""0x9101eda106a443a0fa82375936d0…","""0xd741f826c60dacebb8835278e0f1…","""0xf951e335afb289353dc249e82926…","""sweth""",1.7068e7,2025-03-16 18:00:00,null,"""Mev-commit""",2065.65,3.5256e10,"""Chorus One"""


In [5]:
prices

collateral_asset_address,collateral_asset_price
str,f64
"""0x9d39a5de30e57443bff2a8307a42…",1.16
"""0xf1c9acdc66974dfb6decb12aa385…",1988.93
"""0xf951e335afb289353dc249e82926…",2070.97
"""0xd38bb40815d2b0c2d2c866e0c72c…",0.060421
"""0x20157dbabb84e3bbfe68c349d0d4…",83264.0
"""0xd5f7838f5c461feff7fe49ea5eba…",2025.89
"""0xc02aaa39b223fe8d0a0e5c4f27ea…",1909.49
"""0x2260fac5e5542a773aa44fbcfedf…",83546.0
"""0xfc385a1df85660a7e041423db512…",0.04611


In [17]:
df_staking_data.filter(pl.col("collateral_asset_price_right").is_null()).select(pl.col("collateral_asset_address")).row(0)[0]


'0x78c6b27be6db520d332b1b44323f94bc831f5e33'